# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [79]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [8]:
vaca_df=pd.read_csv("output_data/output.csv")
vaca_df.head()


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Bethel,41.3712,-73.4140,26.01,35,90,4.61,US,1613148956
1,Qaanaaq,77.4840,-69.3632,2.73,71,0,7.09,GL,1613148956
2,Galesong,-5.3166,119.3661,77.00,100,20,4.61,ID,1613148956
3,Atuona,-9.8000,-139.0333,78.35,81,19,13.80,PF,1613148956
4,Ushuaia,-54.8000,-68.3000,59.00,36,40,17.27,AR,1613148956


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [17]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

locations = vaca_df[["Lat", "Lng"]].astype(float)
Humid = vaca_df["Humidity"].astype(float)

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=Humid, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [107]:
hotel_df=vaca_df.loc[(vaca_df['Max Temp']>70) & (vaca_df['Max Temp']<80) & (vaca_df['Wind Speed']<10)
                       & (vaca_df['Cloudiness']==0) & (vaca_df['Humidity']>30)].copy()

hotel_df


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
125,Navsari,20.9524,72.9324,71.60,56,0,4.03,IN,1613148980
138,Māngrol,21.1167,70.1167,70.61,73,0,6.98,IN,1613148982
174,Doha,25.2867,51.5333,71.60,40,0,9.22,QA,1613148735
193,Poum,-20.2333,164.0167,78.73,72,0,4.94,NC,1613148992
230,La Reforma,25.0833,-108.0500,73.00,55,0,1.01,MX,1613148998
275,Malanje,-9.5402,16.3410,71.20,82,0,5.48,AO,1613149006
367,Carahue,-38.7000,-73.1667,73.40,60,0,5.75,CL,1613149023
370,Umluj,25.0213,37.2685,76.73,65,0,7.16,SA,1613149024
414,Grand Gaube,-20.0064,57.6608,75.99,78,0,3.00,MU,1613149032
426,Salalah,17.0151,54.0924,77.00,53,0,4.61,OM,1613149034


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [120]:
hotel_df['Hotel Name']=""
hotel_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
125,Navsari,20.9524,72.9324,71.60,56,0,4.03,IN,1613148980,
138,Māngrol,21.1167,70.1167,70.61,73,0,6.98,IN,1613148982,
174,Doha,25.2867,51.5333,71.60,40,0,9.22,QA,1613148735,
193,Poum,-20.2333,164.0167,78.73,72,0,4.94,NC,1613148992,
230,La Reforma,25.0833,-108.0500,73.00,55,0,1.01,MX,1613148998,


In [126]:
# set up a parameters dictionary
params = {
    "location": "",
    #"keyword": 'Hotel',
    "radius": 5000,
    "type": 'lodging',
    "key": g_key
}

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
for i in range (len(hotel_df)):
    params['location']= f'{hotel_df.iloc[i,1]},{hotel_df.iloc[i,2]}'
# run a request using our params dictionary
    response = requests.get(base_url, params=params)
    hotel_data = response.json()
    hotel_df.iloc[i,-1]= hotel_data['results'][0]['name']

In [128]:
# Print the json (pretty printed)
#print(json.dumps(hotel_data, indent=4, sort_keys=True))

In [130]:
hotel_df.to_csv("output_data/vaca_output.csv")
hotel_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
125,Navsari,20.9524,72.9324,71.60,56,0,4.03,IN,1613148980,Circuit House
138,Māngrol,21.1167,70.1167,70.61,73,0,6.98,IN,1613148982,Hotel The Royal Honours
174,Doha,25.2867,51.5333,71.60,40,0,9.22,QA,1613148735,Doha Marriott Hotel
193,Poum,-20.2333,164.0167,78.73,72,0,4.94,NC,1613148992,Gîte kajeon
230,La Reforma,25.0833,-108.0500,73.00,55,0,1.01,MX,1613148998,Arremar Angostura


In [143]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [139]:
# Add marker layer ontop of heat map
marker_locations = hotel_df[['Lat', 'Lng']]

fig = gmaps.figure()
markers = gmaps.marker_layer(marker_locations,
    info_box_template)
fig.add_layer(markers)
fig



Figure(layout=FigureLayout(height='420px'))

In [ ]:
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>